# Runway Detection
Detect departure and arrival runway from Flight Data Monitoring dataset. Using KNN model to match longitude and latitude of the runway.

## Import 

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd
import math
import numpy as np
import seaborn as sns
import matplotlib as plt
import folium
from sklearn.neighbors import KNeighborsClassifier

### Constant

In [5]:
base_path = '../dataset/database_local'
# base_path = '../dataset/database'

In [6]:
fdm_database_path = base_path + '/01_fdm_files.csv'
fdm_locations_path = base_path + '/02_fdm_locations.csv'
flights_path = base_path + '/03_flights_airport.csv'
landing_output = base_path + '/04_landing_count_detail.csv'
flights_runway_output = base_path + '/04_flights_with_runway.csv'

## Extract Data

#### 1. Read Flights Data 

In [7]:
flights = pd.read_csv(flights_path)
display(flights.head())
flights.shape

,fname,tail_id,dep_airport,arr_airport,fsize_mb,fullpath
0,686200104121532.csv,tail_686_1,BHM,DTW,68.39,../dataset/fdm/tail_686_1/686200104121532.csv
1,686200104121245.csv,tail_686_1,DTW,BHM,61.52,../dataset/fdm/tail_686_1/686200104121245.csv
2,686200104121825.csv,tail_686_2,DTW,LEX,40.71,../dataset/fdm/tail_686_2/686200104121825.csv
3,686200104130429.csv,tail_686_2,LEX,DTW,42.63,../dataset/fdm/tail_686_2/686200104130429.csv
4,686200104130652.csv,tail_687_1,DTW,OKC,95.92,../dataset/fdm/tail_687_1/686200104130652.csv


(5, 6)

In [8]:
flights_clean = flights.dropna().copy()
flights_clean.shape

(5, 6)

#### 2. Read Flight Location

In [9]:
fdm_locations = pd.read_csv(fdm_locations_path)
fdm_locations.head()

,fname,dep_lat,dep_lon,arr_lat,arr_lon
0,686200104121532.csv,0.585762,-1.514118,0.736623,-1.454892
1,686200104121245.csv,0.736719,-1.454814,0.585786,-1.514111
2,686200104121825.csv,0.736698,-1.454781,0.663881,-1.476550
3,686200104130429.csv,0.663908,-1.476544,0.736575,-1.454951
4,686200104130652.csv,0.736596,-1.454951,0.617748,-1.703400


In [10]:
fdm_locations_clean = fdm_locations.dropna().copy()
fdm_locations_clean.shape

(5, 5)

#### 3. Read Runway Data

In [11]:
runways_path = '../dataset/database/runways (radians).xlsx' 
runways = pd.read_excel(runways_path, index_col=0)
runways.head()

,icao_code,iata_code,name,length_ft,width_ft,surface,lighted,closed,le_ident,le_latitude_rad,le_longitude_rad,le_elevation_ft,le_heading_degT,le_displaced_threshold_ft,he_ident,he_latitude_rad,he_longitude_rad,he_elevation_ft,he_heading_degT,he_displaced_threshold_ft
no,,,,,,,,,,,,,,,,,,,,
1,5A8,WKK,Aleknagik / New Airport,2040.0,80.0,GRVL-DIRT-F,0,0,15,NaN,NaN,NaN,NaN,NaN,33,NaN,NaN,NaN,NaN,NaN
2,AGGH,HIR,Honiara International Airport,7218.0,148.0,ASP,1,0,06,-0.164615,2.793312,28.0,68.0,NaN,24,-0.164484,2.793644,14.0,248.0,NaN
3,AGGM,MUA,Munda Airport,4593.0,98.0,ASP,0,0,07,-0.145372,2.744652,7.0,78.9,NaN,25,-0.145329,2.744862,9.0,258.9,NaN
4,AHJ,AHJ,Hongyuan Airport,11800.0,150.0,CON,1,0,16,0.568052,1.786292,11571.0,163.0,NaN,34,0.567513,1.786484,11587.0,343.0,NaN
5,ANYN,INU,Nauru International Airport,7054.0,148.0,ASP,1,0,12,-0.009445,2.913164,22.0,130.3,165.0,30,-0.009665,2.913408,20.0,310.3,380.0


#### 4. Read Airport Data

In [12]:
airports_filepath = '../dataset/database/airports (radians).xlsx'
airports = pd.read_excel(airports_filepath, index_col=0)
airports.head()

,icao_code,iata_code,type,name,latitude_rad,longitude_rad,elevation_ft,scheduled_service
no,,,,,,,,
1,5A8,WKK,medium_airport,Aleknagik / New Airport,1.034677,-2.768406,66.0,yes
2,AF-0005,NaN,medium_airport,Khost International Airport (U.C.),0.580926,1.218368,4204.0,no
3,AGGH,HIR,medium_airport,Honiara International Airport,-0.164550,2.793487,28.0,yes
4,AGGM,MUA,medium_airport,Munda Airport,-0.145350,2.744757,10.0,yes
5,AHJ,AHJ,medium_airport,Hongyuan Airport,0.567782,1.786384,11600.0,no


## Transform Data

#### 1. Concat Runway Identifier

In [13]:
runways_le_ident = runways[['iata_code', 'le_ident', 'le_latitude_rad', 'le_longitude_rad']].copy()
runways_he_ident = runways[['iata_code', 'he_ident', 'he_latitude_rad', 'he_longitude_rad']].copy()

In [14]:
# Rename column to be the same
le_col_rename = {'le_ident': 'runway_ident', 'le_latitude_rad': 'lat_rad', 'le_longitude_rad': 'lon_rad'}
he_col_rename = {'he_ident': 'runway_ident', 'he_latitude_rad': 'lat_rad', 'he_longitude_rad': 'lon_rad'}

runways_le_ident = runways_le_ident.rename(columns=le_col_rename)
runways_he_ident = runways_he_ident.rename(columns=he_col_rename)

runways_le_ident['type'] = 'le'
runways_he_ident['type'] = 'he'

In [15]:
display(runways_le_ident.head())
display(runways_he_ident.head())

,iata_code,runway_ident,lat_rad,lon_rad,type
no,,,,,
1,WKK,15,NaN,NaN,le
2,HIR,06,-0.164615,2.793312,le
3,MUA,07,-0.145372,2.744652,le
4,AHJ,16,0.568052,1.786292,le
5,INU,12,-0.009445,2.913164,le


,iata_code,runway_ident,lat_rad,lon_rad,type
no,,,,,
1,WKK,33,NaN,NaN,he
2,HIR,24,-0.164484,2.793644,he
3,MUA,25,-0.145329,2.744862,he
4,AHJ,34,0.567513,1.786484,he
5,INU,30,-0.009665,2.913408,he


In [16]:
# Concat
runways_frames = [runways_le_ident, runways_he_ident]
runways_ident = pd.concat(runways_frames, ignore_index=False)
runways_ident = runways_ident.sort_index().reset_index().rename(columns={'no': 'rwy_no'})
runways_ident.head()

,rwy_no,iata_code,runway_ident,lat_rad,lon_rad,type
0,1,WKK,15,NaN,NaN,le
1,1,WKK,33,NaN,NaN,he
2,2,HIR,06,-0.164615,2.793312,le
3,2,HIR,24,-0.164484,2.793644,he
4,3,MUA,07,-0.145372,2.744652,le


#### 2. Create New Runway ID with IATA Code

In [17]:
rwy_aprt_id = runways_ident['iata_code'] + '.' + runways_ident['runway_ident']
runways_ident['rwy_aprt_id'] = rwy_aprt_id

In [18]:
runways_ident.head(5)

,rwy_no,iata_code,runway_ident,lat_rad,lon_rad,type,rwy_aprt_id
0,1,WKK,15,NaN,NaN,le,WKK.15
1,1,WKK,33,NaN,NaN,he,WKK.33
2,2,HIR,06,-0.164615,2.793312,le,HIR.06
3,2,HIR,24,-0.164484,2.793644,he,HIR.24
4,3,MUA,07,-0.145372,2.744652,le,MUA.07


## Train Model

In [19]:
cleaned_runways = runways_ident.dropna().copy()

In [20]:
knn = KNeighborsClassifier(n_neighbors=1)

X_train = cleaned_runways[['lat_rad', 'lon_rad']]
y_train = cleaned_runways['rwy_aprt_id']

knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

## Predict Runway

In [21]:
departure_list = knn.predict(fdm_locations_clean[['dep_lat', 'dep_lon']])
departure_list

array(['BHM.36', 'DTW.03L', 'DTW.03L', 'LEX.22', 'DTW.09R'], dtype=object)

In [22]:
arrival_list = knn.predict(fdm_locations_clean[['arr_lat', 'arr_lon']])
arrival_list

array(['DTW.09R', 'BHM.24', 'LEX.22', 'DTW.09R', 'OKC.31'], dtype=object)

#### Merge with FDM filename column

In [23]:
flights_runway = fdm_locations_clean[['fname']].copy()
flights_runway['dep_runway'] = departure_list
flights_runway['arr_runway'] = arrival_list
flights_runway[['dep_aprt', 'dep_rwy']] = flights_runway['dep_runway'].str.split('.',expand=True)
flights_runway[['arr_aprt', 'arr_rwy']] = flights_runway['arr_runway'].str.split('.',expand=True)
flights_runway.drop(columns=['dep_runway', 'arr_runway'], inplace=True)

In [24]:
flights_runway.head()

,fname,dep_aprt,dep_rwy,arr_aprt,arr_rwy
0,686200104121532.csv,BHM,36,DTW,09R
1,686200104121245.csv,DTW,03L,BHM,24
2,686200104121825.csv,DTW,03L,LEX,22
3,686200104130429.csv,LEX,22,DTW,09R
4,686200104130652.csv,DTW,09R,OKC,31


## Check Airport Result
Check if airport result from module 03 is the same with the result

In [25]:
flights_airport = flights_clean[['fname', 'dep_airport', 'arr_airport']]
flights_joined = flights_airport.join(flights_runway.set_index('fname'), on='fname', how='outer')
flights_joined.head()

,fname,dep_airport,arr_airport,dep_aprt,dep_rwy,arr_aprt,arr_rwy
0,686200104121532.csv,BHM,DTW,BHM,36,DTW,09R
1,686200104121245.csv,DTW,BHM,DTW,03L,BHM,24
2,686200104121825.csv,DTW,LEX,DTW,03L,LEX,22
3,686200104130429.csv,LEX,DTW,LEX,22,DTW,09R
4,686200104130652.csv,DTW,OKC,DTW,09R,OKC,31


In [26]:
len(flights_joined[flights_joined.any(1).isnull()])

0

In [27]:
sum(flights_joined['dep_airport'] != flights_joined['dep_aprt'])

0

In [28]:
flights_joined[flights_joined['dep_airport'] != flights_joined['dep_aprt']]

,fname,dep_airport,arr_airport,dep_aprt,dep_rwy,arr_aprt,arr_rwy


In [29]:
sum(flights_joined['arr_airport'] != flights_joined['arr_aprt'])

0

In [30]:
flights_joined[flights_joined['arr_airport'] != flights_joined['arr_aprt']]

,fname,dep_airport,arr_airport,dep_aprt,dep_rwy,arr_aprt,arr_rwy


In [31]:
# Set runway to null if airport result is diffeent
dep_rwy = flights_joined.apply(lambda df: df['dep_rwy'] if df['dep_airport'] == df['dep_aprt'] else None, axis=1)
arr_rwy = flights_joined.apply(lambda df: df['arr_rwy'] if df['arr_airport'] == df['arr_aprt'] else None, axis=1)
flights_joined['dep_rwy'] = dep_rwy
flights_joined['arr_rwy'] = arr_rwy

In [32]:
flights_joined[flights_joined.isna().any(1)]

,fname,dep_airport,arr_airport,dep_aprt,dep_rwy,arr_aprt,arr_rwy


In [33]:
flights_appended = flights.join(flights_joined[['fname', 'dep_rwy', 'arr_rwy']].set_index('fname'),
                                on='fname',
                                how='outer')
col_order = ['fname', 'tail_id', 'dep_airport', 'dep_rwy', 'arr_airport', 'arr_rwy', 'fsize_mb', 'fullpath']
flights_appended = flights_appended[col_order]

## Load to CSV

In [34]:
flights_appended.head()

,fname,tail_id,dep_airport,dep_rwy,arr_airport,arr_rwy,fsize_mb,fullpath
0,686200104121532.csv,tail_686_1,BHM,36,DTW,09R,68.39,../dataset/fdm/tail_686_1/686200104121532.csv
1,686200104121245.csv,tail_686_1,DTW,03L,BHM,24,61.52,../dataset/fdm/tail_686_1/686200104121245.csv
2,686200104121825.csv,tail_686_2,DTW,03L,LEX,22,40.71,../dataset/fdm/tail_686_2/686200104121825.csv
3,686200104130429.csv,tail_686_2,LEX,22,DTW,09R,42.63,../dataset/fdm/tail_686_2/686200104130429.csv
4,686200104130652.csv,tail_687_1,DTW,09R,OKC,31,95.92,../dataset/fdm/tail_687_1/686200104130652.csv


In [35]:
flights_appended.to_csv(flights_runway_output, index=False)

## Find Most Runway Occurance

In [36]:
runways_merged = flights_appended.arr_airport + '.' + flights_appended.arr_rwy
runways_merged.value_counts()

DTW.09R    2
LEX.22     1
BHM.24     1
OKC.31     1
dtype: int64

In [37]:
most_landing = runways_merged.value_counts().index[0]
most_landing_aprt, most_landing_rwy = most_landing.split('.')

In [38]:
most_landing_aprt, most_landing_rwy

('DTW', '09R')

### Create DataFrame

In [39]:
# Create DataFrame
landing_count = pd.DataFrame(runways_merged.value_counts().reset_index())
landing_count.columns = ['airport_runway', 'flight_count']
landing_count.head()

,airport_runway,flight_count
0,DTW.09R,2
1,LEX.22,1
2,BHM.24,1
3,OKC.31,1


In [40]:
airport_runway = landing_count["airport_runway"].str.split(".", n = 1, expand = True)
landing_count["iata_code"] = airport_runway[0]
landing_count["runway_ident"] = landing_count["airport_runway"]

In [41]:
landing_count.columns

Index(['airport_runway', 'flight_count', 'iata_code', 'runway_ident'], dtype='object')

In [42]:
col_order = ['iata_code', 'runway_ident', 'flight_count']
landing_count = landing_count[col_order]

In [43]:
landing_count.head()

,iata_code,runway_ident,flight_count
0,DTW,DTW.09R,2
1,LEX,LEX.22,1
2,BHM,BHM.24,1
3,OKC,OKC.31,1


### Merge with Airport Data 

In [44]:
landing_count_detail = landing_count.join(airports.set_index('iata_code'),
                                          on='iata_code',
                                          how='left')

In [45]:
landing_count_detail.head()

,iata_code,runway_ident,flight_count,icao_code,type,name,latitude_rad,longitude_rad,elevation_ft,scheduled_service
0,DTW,DTW.09R,2,KDTW,large_airport,Detroit Metropolitan Wayne County Airport,0.736745,-1.454791,645.0,yes
1,LEX,LEX.22,1,KLEX,large_airport,Blue Grass Airport,0.663862,-1.476652,979.0,yes
2,BHM,BHM.24,1,KBHM,large_airport,Birmingham-Shuttlesworth International Airport,0.585783,-1.514134,650.0,yes
3,OKC,OKC.31,1,KOKC,large_airport,Will Rogers World Airport,0.617726,-1.703454,1295.0,yes


In [46]:
landing_count_detail.to_csv(landing_output, index=False)

## Visualize Runway

In [47]:
flight_msp_30r =flights_appended[(flights_appended['arr_airport'] == 'MSP') & 
                                 (flights_appended['arr_rwy'] == '30R')]

#### Get 5 Most Departure Airport to MSP.30R 

In [48]:
top_5_airport = flight_msp_30r['dep_airport'].value_counts()[:5].index
top_5_airport = list(top_5_airport)
print(top_5_airport)

[]


#### Get Coordinate for each Airport 

In [49]:
coordinate_cols = ['iata_code', 'latitude_rad', 'longitude_rad']
top_5_coordinate = airports[airports['iata_code'].isin(top_5_airport)][coordinate_cols]

arrival_coordinate = runways[(runways.iata_code == 'MSP') & (runways.he_ident == '30R')]
arr_lat, arr_lon = arrival_coordinate[['he_latitude_rad', 'he_longitude_rad']].iloc[0].values

top_5_coordinate[['arr_lat', 'arr_lon']] = arr_lat, arr_lon
top_5_coordinate = top_5_coordinate.rename(columns={'latitude_rad': 'dep_lat', 'longitude_rad': 'dep_lon'})
top_5_coordinate = top_5_coordinate.set_index('iata_code', drop=True)
top_5_coordinate

,dep_lat,dep_lon,arr_lat,arr_lon
iata_code,,,,


In [50]:
top_5_records = top_5_coordinate.to_records()
top_5_records

rec.array([],
          dtype=[('iata_code', 'O'), ('dep_lat', '<f8'), ('dep_lon', '<f8'), ('arr_lat', '<f8'), ('arr_lon', '<f8')])

#### Visualize Using Map

In [51]:
flight_msp_30r_loc = fdm_locations[fdm_locations.fname.isin(flight_msp_30r.fname)]
flight_msp_30r_loc = flight_msp_30r_loc.set_index('fname')
flight_msp_30r_loc[:5]

,dep_lat,dep_lon,arr_lat,arr_lon
fname,,,,


In [52]:
msp_coordinate = airports[airports['iata_code'] == 'MSP'][['latitude_rad', 'longitude_rad']].iloc[0].values
msp_coordinate = [math.degrees(coor) for coor in msp_coordinate]
print(msp_coordinate)

[44.882, -93.221802]


In [53]:
msp_runways_coordinate = runways_ident[runways_ident['iata_code'] == 'MSP']
msp_runways_coordinate

,rwy_no,iata_code,runway_ident,lat_rad,lon_rad,type,rwy_aprt_id
5628,2815,MSP,22,0.783541,-1.626792,he,MSP.22
5629,2815,MSP,04,0.783169,-1.627315,le,MSP.04
5630,2816,MSP,12L,0.783529,-1.627013,le,MSP.12L
5631,2816,MSP,30R,0.783326,-1.626542,he,MSP.30R
5632,2817,MSP,12R,0.783440,-1.627242,le,MSP.12R
5633,2817,MSP,30L,0.783190,-1.626668,he,MSP.30L
5634,2818,MSP,17,0.783440,-1.627383,le,MSP.17
5635,2818,MSP,35,0.783063,-1.627286,he,MSP.35


In [54]:
arr_coordinate = flight_msp_30r_loc[['arr_lat', 'arr_lon']].values
arr_coordinate = [(math.degrees(coor[0]), math.degrees(coor[1])) for coor in arr_coordinate]
arr_coordinate[:5]

[]

In [55]:
m = folium.Map(location=msp_coordinate, zoom_start=14, tiles="Stamen Terrain")

for coor in arr_coordinate:
    folium.Circle(
        radius=1,
        location=coor,
        color="blue",
        fill=False,
    ).add_to(m)

for row in msp_runways_coordinate.itertuples():    
    folium.Marker(
        location=(math.degrees(row.lat_rad), math.degrees(row.lon_rad)),
        tooltip=f"Runway {row.rwy_aprt_id} ({row.type})",
        popup=f"Runway {row.rwy_aprt_id} ({row.type})",
        icon=folium.Icon(color="green"),
    ).add_to(m)

m